# Dashboard | P6 - Améliorez le produit IA de votre start-up

<img src='https://user.oc-static.com/upload/2019/10/22/15717355057511_Capture%20d%E2%80%99e%CC%81cran%202019-10-22%20a%CC%80%2011.07.21.png'>


# Traitement et analyse des commentaires

In [3]:
%%time
from functions import *
import warnings
import urllib
from PIL import Image
warnings.filterwarnings('ignore')
from tensorflow import keras
model_load = keras.models.load_model('utile/new_model')

@widgets.interact(data = ['Data saved','Collect now'])
def run(data):
    pyLDAvis.enable_notebook()
    
    if data == 'Data saved':
        df = pd.read_csv('utile/df_prétraitement.csv')
        with open('utile/tokens', 'rb') as f1:tokens = pickle.load(f1)
            
        @widgets.interact(topics = (2,5,1))
        def num_topics(topics):
            traitement_txt(df, tokens,topics)
            
            
    else :
        data_collect()
        @widgets.interact(Labelisation = [False,True])
        def run(Labelisation):
            if Labelisation:
                data= pd.read_csv('utile/data_colected.csv')

                @widgets.interact(name = data.name.unique(), n = (0,2,1))
                def run3(name,n):
                    classe = ['drink', 'food', 'interior', 'menu', 'outside']
                    api_key = "PQg94HLFE1DrrNWvV4wqx2FbtCjPwAQtpG8h5tOJcA7p2t3tPmziYJi27gNXmdD5L8qp9WxAd83tvoRLtuggr3tzofjAz2GARhezUc1l_VN3Fv-x1bW9Q5KHCwWxYXYx"
                    yelp_api = YelpAPI(api_key,timeout_s=10)
                    photo = yelp_api.business_query(data[data.name == name].business_id.unique()[0])['photos']

                    req = urllib.request.urlopen(photo[n])
                    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
                    img = cv2.imdecode(arr, 0)
                    traitement(img,True)
                    
                    img_ = cv2.imdecode(arr, 1)
                    image = Image.fromarray(img_.astype(np.uint8))

                    print('Labélisation automatique : ',
                          classe[model_load.predict(np.array([np.array(image.resize((200,200)))])).argmax()])

interactive(children=(Dropdown(description='data', options=('Data saved', 'Collect now'), value='Data saved'),…

CPU times: user 1min 32s, sys: 21.1 s, total: 1min 53s
Wall time: 27.9 s


# Traitement des images

In [11]:
data= pd.read_csv('utile/data_colected.csv')
@widgets.interact(name = data.name.unique(), n = (0,2,1))
def run3(name,n):
    
    classe = ['drink', 'food', 'interior', 'menu', 'outside']
    api_key = "PQg94HLFE1DrrNWvV4wqx2FbtCjPwAQtpG8h5tOJcA7p2t3tPmziYJi27gNXmdD5L8qp9WxAd83tvoRLtuggr3tzofjAz2GARhezUc1l_VN3Fv-x1bW9Q5KHCwWxYXYx"
    yelp_api = YelpAPI(api_key,timeout_s=10)
    photo = yelp_api.business_query(data[data.name == name].business_id.unique()[0])['photos']

    req = urllib.request.urlopen(photo[n])
    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
    img = cv2.imdecode(arr, 0)
    traitement(img,True)

    img_ = cv2.imdecode(arr, 1)
    image = Image.fromarray(img_.astype(np.uint8))

    print('Labélisation automatique : ',
          classe[model_load.predict(np.array([np.array(image.resize((200,200)))])).argmax()])

interactive(children=(Dropdown(description='name', options=('Manteigaria', 'Cervejaria Ramiro', 'Pastéis de Be…

# Segmentation des données images

In [7]:
@widgets.interact(data = ['SIFT','Transfert'],)
def run2(data):   
    df = pd.read_csv('utile/data_photo.csv')
    
    if data == 'SIFT':
        with open('utile/im_features', 'rb') as f1:im_features = pickle.load(f1)
        df_tsne, X_tsne = show_tsne(im_features, df)
        show_kmeans(df_tsne, X_tsne,df)
        
    if data == 'Transfert':
        with open('utile/im_features_tensor', 'rb') as f1:im_features = pickle.load(f1)
        df_tsne, X_tsne = show_tsne(im_features, df)
        show_kmeans(df_tsne, X_tsne,df)

interactive(children=(Dropdown(description='data', options=('SIFT', 'Transfert'), value='SIFT'), Output()), _d…